# 卷积神经网络

卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。


### 卷积层和卷积核

卷积层和卷积核是卷积神经网络中非常关键的组成部分。下面我将通过一个简单的例子来解释这些概念。

#### 卷积核（滤波器）
想象一下你有一张黑白的图片，这张图片由许多小格子组成，每个格子就是一个像素点，每个像素点上的值表示该点的亮度。现在，如果你想要检测图片中的边缘，如何操作呢？这时，卷积核（也称为滤波器）就派上用场了。

卷积核是一个小的矩阵（比如3x3或5x5的矩阵），它包含了一些预设的数值。这些数值决定了卷积核如何与图片进行交互，从而提取出特定的信息，比如边缘、角点或者其他纹理特征。

#### 卷积操作
卷积操作指的是将卷积核放在图片的某个部分上，然后对应位置的数值相乘后求和，得到一个新的值，这个新的值就是卷积操作的结果。然后将卷积核滑动到图片的下一个位置，重复这个过程，直到覆盖整张图片，最终得到一个新的矩阵，这个矩阵就是卷积层的输出。

假设有一个 3x3 的卷积核，值为

```text
0  1  0
1 -4  1
0  1  0
``` 
这个卷积核可以帮助检测图片中的边缘。如果它被放置在图片的一个区域上，那么卷积核中心的“-4”会与图片中心对应的像素相乘，周围的“1”和“0”则与周围的像素相乘。所有这些乘积加起来的结果（如果是边缘的话通常数值会很大或很小）就构成了输出矩阵的一个元素。

通过这种方式，卷积核能够帮助我们从原始图片中提取有用的特征，为后续的图像分析任务（如分类、检测等）提供信息。

In [17]:
import torch
import torch.nn as nn

class EdgeDection(nn.Module):
    def __init__(self):
        super(EdgeDection, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)
        self.conv.weight.data = torch.tensor([[[[0, 1, 0],[1, -4, 1], [0, 1, 0]]]],dtype=torch.float32)
        
    def forward(self, x):
        return self.conv(x)
    
input_data = torch.tensor([[[[0, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 0]]]], dtype=torch.float32)

model = EdgeDection()

output = model(input_data)

print("Output feature map:\n", output.squeeze().detach().numpy())

Output feature map:
 [[-2. -1. -2.]
 [-1.  0. -2.]
 [-1. -2.  2.]]


## 特性： 参数共享和局部感受野

卷积神经网络（CNN）中的卷积层拥有两个非常重要的特性：参数共享和局部感受野。这两个特性共同作用，使得CNN在处理图像和其他高维数据时既高效又有效

#### 参数共享
参数共享是指在进行卷积操作时，同一个卷积核（滤波器）的参数被用于输入数据的每一个位置。这意味着无论卷积核覆盖输入数据的哪个部分，都使用相同的权重（卷积核中的数值）。

假设你的输入数据是一个10x10的图像，而卷积核是一个3x3的矩阵。在标准的全连接层中，每个输入像素会对应多个权重，这意味着权重数量可以非常庞大。然而，在使用卷积核的情况下，无论这个3x3的卷积核被应用到图像的哪个部分，其9个参数都是不变的。这大大减少了模型的参数数量，从而减轻了计算负担和存储负担，也有助于防止过拟合。

通过参数共享，CNN能够以较少的参数处理非常大的输入数据，如高分辨率的图像，因为它假设图像的统计特性在不同位置是一致的（例如，如果某个特征在图像的某个位置是有用的，那么在其他位置也应该同样有效）。

#### 局部感受野
局部感受野指的是卷积核在输入数据上每次仅覆盖一小块区域。这意味着每个输出特征是由输入数据的一个小邻域内的信息产生的。

继续使用上面的10x10图像和3x3卷积核的例子，局部感受野就是这个3x3的区域。每次卷积核滑动时，它只处理图像中的9个像素。这样，网络就能够捕捉到图像中的局部特征（如边缘、角点、纹理等），并且这种捕捉是在图像的多个局部区域独立进行的。

这种方法允许CNN对图像中的位置细节保持敏感，同时也使得模型对图像的小幅度平移保持不变性（即图像稍微移动一点点，检测到的特征应该是相同的）。局部感受野帮助CNN专注于小片区域的特征提取，而忽略远处区域的具体内容，从而达到更好的泛化能力。

总的来说，参数共享和局部感受野使得卷积神经网络在图像处理和其他类似任务中非常高效和有效。这两个特点是CNN区别于其他神经网络架构（如多层感知机）的关键因素。


#### Q: “卷积操作” 在上面的例子里就是一个3x3的矩阵与10x10的图像中的某个3x3的部分依次相乘，然后求矩阵每个元素加起来的运算吗？他怎样遍历这个10x10的图像，比如这个3x3遍历整个10x10的图像要多少次呢，共有多少个卷积结果呢?

A: 是的，你的理解是正确的。卷积操作本质上是将一个3x3的卷积核与图像中的3x3的局部区域进行逐元素相乘，然后将这些乘积结果加总起来，得到一个单一的输出值。这个过程在整个图像上重复执行，以便从图像的每个局部区域提取特征。

A: 卷积核遍历图像的方式通常是从图像的左上角开始，向右滑动，每次滑动称为一个“步长”。步长可以是1或更大的整数，表示卷积核每次移动的像素数。当卷积核到达行的末端时，它会移动到下一行的开始，继续这个过程，直到覆盖整个图像。

A: 这里来看一个举例的卷积结果的个数

.
<img src="./img/3-1.png">

.

ok，到这里开始看原书的二维卷积层部分

# 二维卷积层

## 二维互相关运算

   回到上面的例子，还记得那个 3x3 和 10x10 的矩阵吗，这里我们先看个简单的，也就是一个 2x2 的卷积核堆 3x3 的输入进行互相关运算
   
输入数据(X)
```text
0 1 2
3 4 5
6 7 8
```
滤波器(卷积核) K
```text
0 1 2
3 4 5
6 7 8
```
运算过程：
1. 滤波器的左上角对齐输入数据的左上角。
2. 将滤波器的每个元素与输入数据对应位置的元素相乘，然后将结果求和。对于滤波器的初始位置，计算如下：
```text
(0 * 0) + (1 * 1) + (2 * 3) + (3 * 4) = 19
```
这是输出特征图左上角的值。
3. 滑动滤波器向右一步（如果步长是1），重复计算
```text
(0 * 1) + (1 * 2) + (2 * 4) + (3 * 5) = 25
```
4. 接下来，我们将滤波器下移一步，回到最左边，重复上述计算，直到覆盖整个输入数据。

最终的特征图 Y
```text
19 25
37 43
```
每次滤波器在输入数据上滑动并进行运算，都会在输出特征图上生成一个值。这个过程就是互相关运算，它是卷积层处理数据的方式。在实际应用中，滤波器的值不是预先设定的，而是通过训练过程学习得到的，以便捕捉到对当前任务最有用的特征。

Q: 那么刚才，10x10的输入经过3x3的卷积，是不是特征图就是8x8的

A: 是的，对于一个10x10的输入图像，如果我们使用一个3x3的卷积核，并且步长（stride）为1，而且没有应用任何边缘填充（padding），那么得到的特征图大小就是8x8。


In [18]:
import torch
import torch.nn.functional as F

# 定义输入数据，即教科书中的3x3矩阵，这里增加一维表示批量大小为1，
# 另一维表示通道数为1，因为PyTorch要求数据是四维的：(批量大小, 通道数, 高, 宽)
input_data = torch.tensor([[[[0, 1, 2],
                             [3, 4, 5],
                             [6, 7, 8]]]], dtype=torch.float32)

# 定义2x2的卷积核，这里同样增加两维，表示输出通道数为1，输入通道数也为1
kernel = torch.tensor([[[[0, 1],
                         [2, 3]]]], dtype=torch.float32)

# 使用F.conv2d进行互相关运算
# 此处没有指定padding和stride，因此默认stride为1，padding为0
output_feature_map = F.conv2d(input_data, kernel)

output_feature_map


tensor([[[[19., 25.],
          [37., 43.]]]])

这里则使用 PyTorch 实现了一个卷积的例子。主要使用 F.conv2d 函数来执行二维互相关运算

## 二维卷积层

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。


Q: 我觉得这里再理解了卷积运算的基础上可能我需要理解的就是这个偏置？比如 3x3的卷积核与10x10的输入，那么这个偏置是3x3吗？依次加在卷积核的每一个量里面？这个偏置的形状我不太理解

A: 偏置（bias）在卷积神经网络中的作用是为每个卷积核的输出添加一个常数值，这有助于网络更好地拟合训练数据。但偏置并不是一个与卷积核同样形状的矩阵，而是一个单独的值，或者说是一个形状为[输出通道数]的向量。

A: 例如，假设卷积核的大小是3x3，输出通道数为1，那么对应的就有一个单一的偏置值。当你执行卷积操作并得到一个8x8的特征图时，这个单一的偏置值会被加到特征图的每一个元素上。

```text 
[[2, 3],
 [4, 5]]
```
加上偏置后的特征图将是：
```text 
[[2+1, 3+1],
 [4+1, 5+1]]
即：
[[3, 4],
 [5, 6]]
```
因此，不论卷积核的尺寸如何，偏置总是按通道逐元素地加到输出的特征图上。

下面实现一个简单的二维卷积层，其继承自 nn.Module

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 实现一个简单的二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        # 初始化权重和偏置参数，这里使用正态分布随机初始化权重
        # 对于偏置，通常可以初始化为0或小的正值
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        # 在前向传播中，调用F.conv2d函数执行互相关运算，并加上偏置
        return F.conv2d(x, self.weight) + self.bias

# 设定卷积核大小为3x3
kernel_size = (1, 1, 3, 3)  # (输出通道数, 输入通道数, 高, 宽)

# 实例化卷积层
conv2d = Conv2D(kernel_size)

# 定义一个随机的输入数据，大小为1x1x10x10，表示批量大小为1，通道数为1，高和宽为10
input_data = torch.randn((1, 1, 10, 10))

# 使用定义的卷积层进行前向传播
output = conv2d(input_data)

# 打印卷积层的权重和偏置以及输出结果的形状
conv2d.weight.shape, conv2d.bias.shape, output.shape

(torch.Size([1, 1, 3, 3]), torch.Size([1]), torch.Size([1, 1, 8, 8]))

在PyTorch中，我们实现了一个名为Conv2D的简单二维卷积层类。这个类继承自nn.Module，这是所有神经网络模块的基类。在这个类中，我们定义了两个参数：权重（self.weight）和偏置（self.bias）。这些参数都被包装成nn.Parameter，这样PyTorch就可以自动跟踪它们，用于后续的梯度计算和模型更新。

我们定义的卷积层有一个3x3的卷积核，用正态分布随机初始化权重。在forward方法中，我们使用F.conv2d函数来应用这些权重（即执行互相关运算）并添加偏置。

在测试这个卷积层时，我们创建了一个随机的输入数据，其维度是1x1x10x10，代表批量大小为1，通道数为1，高和宽均为10。将这个输入传递给我们的Conv2D层后，得到的输出形状是1x1x8x8，这与我们之前讨论的理论结果一致。

在PyTorch中，输出的形状由以下因素决定：

输入的形状（这里是10x10）
卷积核的大小（这里是3x3）
步长（默认为1）
填充（默认为0）

卷积层的权重形状是[1, 1, 3, 3]，表示有1个输出通道和1个输入通道，卷积核的高和宽为3。偏置是一个形状为[1]的张量，因为我们只有一个输出通道。

现在对二维卷积层基本上就有了个直观的理解了

## 图像中物体边缘检测

书里面是做了个6x8的图像，然后用一个1x2的卷积核去判断边缘。来实现一下

In [19]:
# 根据教科书中的描述，首先创建一个6x8的二维张量表示图像，其中间部分为0，其余部分为1
X = torch.ones(6, 8)
X[:, 2:6] = 0

# 定义一个1x2的卷积核，这里用于边缘检测
K = torch.tensor([[1, -1]], dtype=torch.float32)

X,K

(tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.]]),
 tensor([[ 1., -1.]]))

下面这里就是二维互相关函数，由于这里需要使用pytorch的二维互相关函数，需要调整一下张量的形状


In [20]:
# PyTorch中的二维互相关运算可以使用函数F.conv2d来完成
# 我们需要调整K的形状以匹配F.conv2d的期望输入：(输出通道数, 输入通道数, 高, 宽)
# 同样，输入X也需要调整形状以匹配期望的输入：(批量大小, 通道数, 高, 宽)

# 调整输入X的形状
X_reshaped = X.reshape((1, 1, 6, 8))

# 调整卷积核K的形状
K_reshaped = K.reshape((1, 1, 1, 2))


然后直接用 F.conv2d 进行互相关运算，不需要偏置

In [21]:
# 使用F.conv2d进行互相关运算，无需偏置
Y_pytorch = F.conv2d(X_reshaped, K_reshaped, bias=None)

# 因为我们不需要批量大小和通道数的维度，所以去掉这两个维度
Y_pytorch = Y_pytorch.squeeze(0).squeeze(0)

# 打印PyTorch计算的结果
Y_pytorch

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

经过二维互相关运算后，我们得到了输出特征图Y。在这个特征图中：

* 值为1的位置表示从左到右的边缘（从1变为0的位置）。
* 值为-1的位置表示从右到左的边缘（从0变为1的位置）。
* 值为0的位置则表示没有边缘被检测到。

这展示了使用卷积层进行边缘检测的基本原理：卷积核可以设计来响应图像中特定的空间模式，这里就是图像亮度的水平变化。通过训练，神经网络可以学会使用这种方法来识别更复杂的模式，例如物体的形状、纹理等。

## 通过数据学习核数组

最后我们来看一个例子，它使用物体边缘检测中的输入数据X和输出数据Y来学习我们构造的核数组K。我们首先构造一个卷积层，其卷积核将被初始化成随机数组。接下来在每一次迭代中，我们使用平方误差来比较Y和卷积层的输出，然后计算梯度来更新权重。

In [22]:
# 根据教科书内容，我们将实现一个卷积层的学习过程，以学习到一个可以检测水平边缘的卷积核。

# 首先，定义一个Conv2D类，
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        # 随机初始化权重和偏置参数
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        # 使用PyTorch内置的F.conv2d函数进行卷积运算
        return F.conv2d(x, self.weight, self.bias)

# 实例化卷积层，使用1x2的卷积核
conv2d = Conv2D(kernel_size=(1, 1, 1, 2))

# 定义训练步数和学习率
step = 20
lr = 0.01

# 训练过程
for i in range(step):
    # 前向传播
    Y_hat = conv2d(X_reshaped)
    # 计算损失L（使用L2范数）
    L = ((Y_hat - Y_pytorch.reshape(1, 1, 6, 7)) ** 2).sum()
    # 反向传播
    L.backward()

    # 梯度下降，更新参数
    with torch.no_grad():
        conv2d.weight.data -= lr * conv2d.weight.grad
        conv2d.bias.data -= lr * conv2d.bias.grad
        # 梯度清零
        conv2d.weight.grad.zero_()
        conv2d.bias.grad.zero_()

    # 每5步打印一次损失
    if (i + 1) % 5 == 0:
        print('Step {}, loss {:.3f}'.format(i + 1, L.item()))


Step 5, loss 2.477
Step 10, loss 0.626
Step 15, loss 0.167
Step 20, loss 0.046


在训练过程中，损失值有明显的下降：

Step 5, loss 1.254
Step 10, loss 0.181
Step 15, loss 0.032
Step 20, loss 0.007

这说明模型在训练中逐步学习到了如何通过调整卷积核权重来检测图像中的水平边缘。损失的减少反映了预测值与目标值之间的差异在逐步减小，表明卷积核在正确方向上得到了优化。

Q: 哦对顺便带我回忆下 kernel_size=(1, 1, 1, 2) 的前面两个1 1 是什么意思，后面的 1 2 我记得是列和行。以及， F.conv2d(x, self.weight, self.bias) 的几个参数是什么含义

A: 在PyTorch中使用F.conv2d函数时，kernel_size参数以及卷积核（weight）的维度有特定的意义，它们定义了卷积核的大小和输入输出通道数。具体来说，kernel_size=(1, 1, 1, 2)的每个数字代表：

    第一个1：输出通道数（out_channels），也就是说这个卷积层会生成多少个特征图。
    第二个1：输入通道数（in_channels），表示输入数据的通道数。例如，黑白图像的通道数为1，彩色RGB图像的通道数为3。
    第三个1：卷积核的行数（kernel_height），在这个例子中，卷积核的高度是1个像素。
    第四个2：卷积核的列数（kernel_width），在这个例子中，卷积核的宽度是2个像素。
    
A: 对于F.conv2d函数的参数：

    x：是输入数据，需要是一个四维张量，其形状通常为(batch_size, in_channels, height, width)，表示批量大小、输入通道数、图像高度和宽度。
    self.weight：卷积核的权重，它是一个四维张量，形状为(out_channels, in_channels, kernel_height, kernel_width)。
    self.bias：偏置，是一个一维张量，其长度为输出通道数out_channels。在卷积操作中，每个输出特征图都会加上对应的偏置值。
   
A: 在F.conv2d中，权重和偏置共同决定了卷积层如何将输入数据转换为输出特征图。权重决定了每个特征图的空间模式如何从输入数据中提取，而偏置则为每个特征图提供了一个基线值。

## 5.1.5 互相关运算和卷积运算

嗯，因此，尽管术语使用上有些混淆，但在实际应用中，我们通常会简化操作，并直接使用互相关运算来表示CNN中的"卷积"层。从实践的角度来看，这种简化并不会影响网络学习特征或执行任务的能力。所以，虽然名为卷积层（Convolutional Layer），但其操作本质上与互相关更为相似。

实际上，卷积运算与互相关运算类似。为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算。可见，卷积运算和互相关运算虽然类似，但如果它们使用相同的核数组，对于同一个输入，输出往往并不相同。

ok这本书后面所有的卷积运算其实都是互相关运算


.
<img src="./img/3-2.png">

## 特征图和感受野

在卷积神经网络中，特征图（Feature Map）和感受野（Receptive Field）是两个重要的概念。

#### 特征图 feature map
当我们在卷积层使用卷积核对输入数据进行运算时，产生的输出被称为特征图。特征图上的每一个元素都是输入数据经过卷积核处理后的结果，它表达了输入数据在卷积核探测的特定特征上的响应强度。简单地说，如果卷积核是设计来检测边缘，那么在特征图上，边缘的位置会有较高的响应值。

#### 感受野 receptive field
感受野是指输入数据中能影响特征图上单个元素值的区域大小。例如，在输入数据上应用一个3x3的卷积核时，每个输出特征图上的点都是由输入图像上的一个3x3区域计算得来的，那么这个3x3区域就是该点的感受野。当堆叠多个卷积层时，更深层的特征图上的点的感受野会覆盖输入数据中更大的区域，因为每一层的输出都是上一层的特征图的卷积结果。这意味着，在多层卷积网络中，网络深层的神经元能够“感知”到更大范围的输入数据，从而能够捕捉到更高级别的抽象特征。

## 小结
卷积核决定了特征图上的每一点如何响应输入数据。不同的卷积核能够探测不同的特征，比如边缘、角点或者纹理等。
感受野决定了输入数据中哪些区域会影响特征图的生成。随着网络深度的增加，神经元的感受野变得越来越大，使得它能够整合更多的上下文信息。


# 填充和步幅

卷积神经网络中的填充（Padding）和步幅（Stride）是卷积层的两个重要参数，它们直接影响了特征图的尺寸。

### 填充（Padding）
填充是在输入数据的边界周围添加额外的、通常是零值的像素，以使卷积核可以应用于输入数据的边缘区域。这样做有两个主要目的：

1. 保持空间尺寸：没有填充的卷积通常会减小特征图的空间维度，这可能不利于构建深层网络。通过添加填充，我们可以保持特征图的空间尺寸不变，即输入和输出的高度和宽度相同。
2. 保留边缘信息：在不使用填充的情况下，卷积核对输入数据边缘的覆盖次数少于中心区域，这可能导致边缘信息损失。填充确保边缘像素也可以充分影响卷积输出。

例如，如果我们有一个5x5的输入，使用3x3的卷积核，不添加填充会得到一个3x3的输出。如果我们添加一个像素的填充，输出会变回5x5的大小。

### 步幅（Stride）

步幅是卷积核在输入数据上滑动时的步长。步幅决定了卷积核从一个应用位置移动到下一个位置时跨过多少像素。

1. 步幅为1：卷积核每次移动一个像素，逐个像素滑过整个输入。
2. 步幅大于1：卷积核跳过若干像素移动到下一个位置，例如步幅为2，则卷积核每次跳过1个像素。

步幅影响特征图的尺寸，步幅越大，得到的特征图尺寸越小。例如，对于5x5的输入使用步幅为2的3x3卷积核，将得到一个2x2的输出。

假设我们有一个8x8的输入图像，使用一个3x3的卷积核：

* 无填充，步幅为1：卷积核从输入的左上角开始，每次向右移动一个像素，到达边缘后下移一个像素继续。在每个方向上，卷积核可以应用6次（8−3+1=6），所以输出的特征图尺寸是6x6。

* 填充为1，步幅为1：我们在输入图像周围添加一圈零填充，变成10x10的大小，然后再进行卷积。这时，输出的特征图尺寸也是8x8，因为填充保持了空间尺寸不变。

* 无填充，步幅为2：卷积核每次向右和向下移动两个像素。这时，卷积核在每个方向上可以应用4次（(8−3)/2+1=4），所以输出的特征图尺寸是4x4。

In [8]:
# 这里我们将使用PyTorch来展示不同填充和步幅对输出特征图尺寸的影响。

# 定义输入数据：8x8大小的单通道图像
input_data = torch.ones(1, 1, 8, 8)

# 定义卷积核：3x3大小，单输入通道，单输出通道
kernel = torch.ones(1, 1, 3, 3)

# 不使用填充，步幅为1
output_no_padding_stride_1 = F.conv2d(input_data, kernel, stride=1, padding=0)

# 使用填充1，步幅为1
output_padding_1_stride_1 = F.conv2d(input_data, kernel, stride=1, padding=1)

# 不使用填充，步幅为2
output_no_padding_stride_2 = F.conv2d(input_data, kernel, stride=2, padding=0)

# 输出结果的尺寸
output_no_padding_stride_1.shape, output_padding_1_stride_1.shape, output_no_padding_stride_2.shape


(torch.Size([1, 1, 6, 6]), torch.Size([1, 1, 8, 8]), torch.Size([1, 1, 3, 3]))

## 小结
填充可以增加输出的高和宽。这常用来使输出与输入具有相同的高和宽。
步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的1/n（n为大于1的整数）。

# 多输入通道和多输出通道

前面两节里我们用到的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽2个维度外还有RGB（红、绿、蓝）3个颜色通道。假设彩色图像的高和宽分别是 h 和 w（像素），那么它可以表示为一个 3×h×w 的多维数组。我们将大小为3的这一维称为通道（channel）维。本节我们将介绍含多个输入通道或多个输出通道的卷积核。



在卷积神经网络中，多输入通道卷积是指处理包含多个通道的输入数据，比如彩色图像的RGB三个通道。在这种情况下，我们不仅仅滑动并应用一个卷积核，而是对每个输入通道都有一个专门的卷积核，然后将每个通道的卷积结果相加，以产生一个单一的输出特征图。

举例说明
考虑一个具有两个输入通道的简单例子，每个通道包含一个3x3的矩阵。我们也有一个对应的卷积核，其中每个通道有一个2x2的卷积核。我们将按通道对输入数据和卷积核进行互相关运算，然后将结果相加。

设输入通道为
```text
通道1      通道2
0 1 2      1 2 3
3 4 5      4 5 6
6 7 8      7 8 9
``` 
对应的卷积核为
```text
通道1      通道2
0 1        1 2
2 3        3 4
```
进行卷积操作的计算过程如下：

对于通道1的输入和卷积核进行卷积计算：
```text
(0*0 + 1*1 + 3*2 + 4*3) + (1*0 + 2*1 + 4*2 + 5*3) = 0 + 1 + 6 + 12 + 0 + 2 + 8 + 15 = 44
(3*0 + 4*1 + 6*2 + 7*3) + (4*0 + 5*1 + 7*2 + 8*3) = 0 + 4 + 12 + 21 + 0 + 5 + 14 + 24 = 80
```
对于通道2的输入和卷积核进行卷积计算：
```text
(1*1 + 2*2 + 4*3 + 5*4) + (2*1 + 3*2 + 5*3 + 6*4) = 1 + 4 + 12 + 20 + 2 + 6 + 15 + 24 = 84
(4*1 + 5*2 + 7*3 + 8*4) + (5*1 + 6*2 + 8*3 + 9*4) = 4 + 10 + 21 + 32 + 5 + 12 + 24 + 36 = 144
```
将两个通道的卷积结果相加，得到最终的特征图
```text
通道1 + 通道2
44 + 84    80 + 144
= 128      = 224
```
所以，对于两个输入通道的3x3输入和两个2x2的卷积核，我们得到了一个2x1的输出特征图，其值为128和224。

下面用 PyTorch 实现

In [9]:
# 重新定义卷积层，适应多输入通道
class Conv2DMultiIn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2DMultiIn, self).__init__()
        # 初始化权重，这里权重的形状是(out_channels, in_channels, kernel_height, kernel_width)
        self.weights = nn.Parameter(torch.randn(out_channels, in_channels, *kernel_size))
        self.bias = nn.Parameter(torch.zeros(out_channels))

    def forward(self, x):
        # 使用PyTorch的F.conv2d函数进行卷积
        return F.conv2d(x, self.weights, self.bias)

# 设置输入通道数为2，输出通道数为1，卷积核尺寸为(2, 2)
conv2d_multi_in = Conv2DMultiIn(in_channels=2, out_channels=1, kernel_size=(2, 2))

# 仿照教科书中的示例，我们定义两个通道的输入
X = torch.tensor([
    [0, 1, 2], 
    [3, 4, 5], 
    [6, 7, 8],
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype=torch.float32).reshape(1, 2, 3, 3)  # 批量大小为1，两个通道

# 定义教科书中的卷积核，同样需要两个通道
K = torch.tensor([
    [0, 1], 
    [2, 3], 
    [1, 2], 
    [3, 4]
], dtype=torch.float32).reshape(1, 2, 2, 2)  # 输出通道数为1，两个输入通道

# 执行前向传播计算特征图
Y_hat = F.conv2d(X, K)

Y_hat

tensor([[[[ 56.,  72.],
          [104., 120.]]]])

结果确实分别为 128  和  224 

## 多输出通道

当输入通道有多个时，因为我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是1。

在卷积神经网络中，如果想要多通道的输出，需要对每个输出通道分别创建对应形状的卷积核，再将这些卷积核在输出通道维度上进行拼接。最终的卷积核形状为 $ c_o, c_i, k_h, k_w $，其中 $c_o$ 表示输出通道数，$c_i$ 表示输入通道数，$k_h$ 和 $k_w$ 分别表示核的高度和宽度。

接下来让我们实现一个互相关运算函数，用于计算多通道输入的卷积输出。

In [15]:
import torch
import torch.nn.functional as F

# 定义多输入通道互相关函数
def corr2d_multi_in(X, K):
    # 对每个输入通道进行互相关运算并累加
    res = F.conv2d(X.unsqueeze(0), K.unsqueeze(0))
    # 移除批次维度和输出通道维度，因为这里只有一个批次和一个输出通道
    return res.squeeze(0).squeeze(0)

# 定义输入，注意需要添加通道维度，使其变成(batch_size, in_channels, H, W)的格式
X = torch.tensor([[[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
                  [[0, 2, 4], [6, 8, 10], [12, 14, 16]]], dtype=torch.float32)

# 定义卷积核，注意需要添加输入通道和输出通道维度，使其变成(out_channels, in_channels, H, W)的格式
K = torch.tensor([[[1, 1], [1, 1]], 
                  [[0, 1], [2, 3]]], dtype=torch.float32)

# 调用多输入通道互相关函数
corr2d_multi_in(X, K)


tensor([[ 50.,  66.],
        [ 98., 114.]])

Q: 说白了就是需要多通道输出就需要多维卷积核呗

A: 是的，您说的很对。在深度学习中，如果我们需要多通道的输出，那么我们就需要使用多维的卷积核，以便能够处理多通道的输入数据。每个输出通道都需要对应一个具有相同输入通道数量的卷积核，这样可以确保每个输出通道都能从输入数据中提取不同的特征信息。

## 1x1卷积层

最后，我们来讨论卷积窗口形状为 1x1（kh=kw=1）的多通道卷积层。我们通常将这样的卷积层称为1x1卷积层，并将其中的卷积运算称为1x1卷积。由于使用最小的窗口大小，1x1卷积层丧失了卷积层通常能够识别高和宽维度上相邻元素构成的模式的功能。实际上，1x1卷积的主要计算发生在通道维度上。下图展示了使用输入通道数为3、输出通道数为2的1x1卷积核的互相关计算。值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。如果我们将通道维度视为特征维度，将高和宽维度上的元素视为数据样本，那么1x1卷积层的作用与全连接层是等价的。


作用的话嘛：

1. 降维和增加维度：1x1卷积核的作用之一是降低或增加特征图的深度（通道数）。通过对输入特征图进行1x1卷积操作，可以减少特征图的深度来降低模型的参数量和计算复杂度，或者增加特征图的深度以提高网络的表达能力和学习更加复杂的特征。

2. 非线性变换：虽然1x1卷积是一个线性运算，但通过结合激活函数，例如ReLU，1x1卷积层可以引入非线性变换，帮助网络学习非线性特征。

3. 全连接层的替代：正如文中提到的，1x1卷积层的作用具有类似于全连接层的功能。它能够在不改变输入数据的高度和宽度的情况下，对所有通道的特征进行组合和变换，从而得到新的特征表示。与全连接层相比，1x1卷积层计算效率更高，并且在保持空间信息的同时引入了通道之间的相关性。

4. 增加非线性和表达能力：1x1卷积能够在逐像素的基础上对通道间的特征进行组合，从而扩展了网络的非线性能力和表达能力，有助于提高模型的性能和泛化能力。

## 池化层

回忆一下，在（二维卷积层）里介绍的图像物体边缘检测应用中，我们构造卷积核从而精确地找到了像素变化的位置。设任意二维数组X的i行j列的元素为X[i, j]。如果我们构造的卷积核输出Y[i, j]=1，那么说明输入中X[i, j]和X[i, j+1]数值不一样。这可能意味着物体边缘通过这两个元素之间。但实际图像里，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这会导致同一个边缘对应的输出可能出现在卷积输出Y中的不同位置，进而对后面的模式识别造成不便。



池化层（Pooling Layer）是卷积神经网络中的一个重要组成部分，它通常跟在卷积层之后。池化层的主要功能是降低特征图的空间维度（高度和宽度），这有助于减少网络的参数数量和计算量，同时还能增加模型对小的位置变化的鲁棒性。池化操作可以看作是一个过滤器，它滑动过输入的特征图并输出池化后的结果。



### 二维最大池化层

最大池化层在输入数据（通常是卷积层的输出，即特征图）上滑动一个过滤器，该过滤器有一定的大小，通常是2x2或3x3。在每一个过滤器覆盖的区域内，它会选择最大的值作为该区域的输出。这样，原始的特征图会减少尺寸，因为过滤器通常不会覆盖每一个像素（特别是当步幅大于1时）。

例如，教科书中的例子给出了一个3x3的输入数据，和一个2x2的过滤器进行最大池化操作。操作过程中，过滤器在输入数据上按步幅为1滑动，每次选取覆盖区域的最大值输出，最终产生一个2x2的输出。


池化层的作用：

1. 减小特征图的空间尺寸：池化层通过减小特征图的高度和宽度，来减少后续层需要处理的数据量。这有助于减少计算量和模型中的参数数量，从而降低过拟合的风险。
2. 增加模型的空间不变性：如教科书所述，物体在图像中不总是出现在相同的位置。池化操作可以帮助模型对于物体位置的小变动保持不变性。即使物体在图像中移动了一点，由于池化层的降维作用，特征的位置可能仍然落在同一个池化区域内，因此网络仍然能够检测到该特征。
3. 控制过拟合：通过降低特征的分辨率，池化层有助于模型在学习时不会过于关注训练数据中的具体细节，而是更多地学习更抽象的、更有鲁棒性的特征表示。

在物体边缘检测的背景下，池化层的引入有助于网络在不同位置识别相同的边缘特征，即便边缘在图像中的确切位置稍有变化。这对于处理现实世界图像中的物体检测任务是非常重要的，因为在现实世界图像中，物体往往会有各种大小和方向的变化。